# How to work with objects in filegroups
In this recipe we'll explore how to create table and index to a filegroup and then how to move the table to some other filegroup.

First of all, let's observe which filegroups we have in our *Demo* database.

In [ ]:
use Demo
go

select * from sys.filegroups;


## Creating a table and an index to different filegroups

In [ ]:
use Demo
go

CREATE TABLE dbo.TestTable
(
Id int not null identity constraint pk_TestTable primary key clustered
, SomeValue dec(10, 3) not null    
) ON [PRIMARY]
go

create index ix_TestTable_SomeValue on dbo.TestTable (SomeValue) ON FG1
go

Let's take a look on both previously created objects.

In [ ]:
select 
    i.name as index_name
    , s.name as filegroup_name
from sys.indexes as i
    join sys.data_spaces as s on i.data_space_id = s.data_space_id
where i.name = 'ix_TestTable_SomeValue'

select 
    i.name as index_name
    , s.name as filegroup_name
from sys.indexes as i
    join sys.data_spaces as s on i.data_space_id = s.data_space_id
where i.name = 'pk_TestTable'

exec sp_help 'dbo.TestTable'

## Moving table to different filegroup
The trick is to move ***clustered index*** of the table between filegroups. Very often, primary key of a table is clustered index. If the table does not have clustered index, we can temporarily create one to the target filegroup. In following code sample we'll move the table and the non-clustered index between filegroups.

In [ ]:
use Demo
go

alter table dbo.TestTable drop constraint pk_TestTable
alter table dbo.TestTable add constraint pk_TestTable primary key (Id) on FG1
go

create index ix_TestTable_SomeValue on dbo.TestTable (SomeValue) with DROP_EXISTING on [PRIMARY]
go

Now, we should be done. Let's review the storages ones again.

In [ ]:
select 
    i.name as index_name
    , s.name as filegroup_name
from sys.indexes as i
    join sys.data_spaces as s on i.data_space_id = s.data_space_id
where i.name = 'ix_TestTable_SomeValue'

select 
    i.name as index_name
    , s.name as filegroup_name
from sys.indexes as i
    join sys.data_spaces as s on i.data_space_id = s.data_space_id
where i.name = 'pk_TestTable'

exec sp_help 'dbo.TestTable'